In [1]:
import pyodbc
from import_function import import_scoring_data, import_data 
from tto_pricing_features import tto_pricing_features 
ct='New'
response='TOTAL_REVENUE_CY'
from features_by_customer_type import customer_type_features
cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=customer_type_features(ct)
cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=tto_pricing_features(ct) 
scoring_data = "SELECT A.* FROM (SELECT * FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2015 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100" 
data = "SELECT * FROM (SELECT * FROM CTG_ANALYTICS_WS.SM_RETENTION_MODEL WHERE TAX_YEAR=2014  and CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX')A order by random() limit 10000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)

c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
import_function.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'] = 0
import_function.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool['paid_something'][df[response] > 0]=1
c:\python27\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A v

      STATE_ATTACH_COUNT  MINDBENDER_FLAG  CA_REFUND_TRANSFER_FLAG  \
0                    1.0              0.0                      0.0   
1                    1.0              0.0                      0.0   
2                    1.0              0.0                      0.0   
3                    1.0              0.0                      0.0   
4                    1.0              0.0                      0.0   
5                    0.0              0.0                      0.0   
6                    0.0              0.0                      0.0   
7                    0.0              0.0                      0.0   
8                    1.0              0.0                      0.0   
9                    0.0              0.0                      0.0   
10                   0.0              0.0                      0.0   
11                   1.0              0.0                      0.0   
12                   1.0              0.0                      0.0   
13                  

import_function.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool.drop([f], axis=1, inplace=True)


In [ ]:
from bin_cont_features import bin_pca
j='True'
response='paid_something'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)


In [ ]:
ct='New'
response='paid_something'
# print dummy_pca.dtypes
from feature_clustering import feature_clustering
i='pca'
df_no_pca=dummy_pca
df_no_pca, y, plsca= feature_clustering(df_no_pca,response, i)
print df_no_pca.shape, y.shape




In [ ]:
from clustering import obs_clustering
clust='True'
df=df_no_pca
print df.shape, list(df)

df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)
print df.shape, df_w_labels.shape, df_w_clust.shape

In [ ]:
from pca_dim_red import pca_code
i='False'
response='paid_something'
df_no_pca=df_w_labels

df_pca, fitted_pca = pca_code(df_no_pca,response,i)
print df_pca.shape



In [ ]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
x, y = get_arrays(dummy_pca, response, i, j)
print x.shape

In [ ]:
from model_algorithm import algorithm
print x.shape
models, name = algorithm(x, y, response)

In [ ]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont